# TODO:
- write data
- add homo/lumo to pretrain
- check number_radical_e
- PRETRAIN: USE SCAFFOLD SPLIT ON PCMM4 TO SELECT ETEROGENEOUS DATA 500K SHOULD BE A REASONABLE TEST AMMOUNT

## Notes from https://ogb.stanford.edu/docs/lsc/pcqm4mv2/

3D Graph: We  provide equilibrium 3D graph structure for training molecules in the form of the SDF format. 
The compressed SDF can be downloaded with the code below (1.5GB). 
The single SDF contains 3D information of all the 3,378,606 training molecules. 
First download and decompress the file as follows.
```
wget http://ogb-data.stanford.edu/data/lsc/pcqm4m-v2-train.sdf.tar.gz
md5sum pcqm4m-v2-train.sdf.tar.gz # fd72bce606e7ddf36c2a832badeec6ab
tar -xf pcqm4m-v2-train.sdf.tar.gz # extracted pcqm4m-v2-train.sdf
```
Then, use rdkit to extract molecule information as follows.

    from rdkit import Chem
    suppl = Chem.SDMolSupplier('pcqm4m-v2-train.sdf')
    for idx, mol in enumerate(suppl):
        print(f'{idx}-th rdkit mol obj: {mol}')


- These structures are calculated by DFT. (cit from tgt: "DFT coordinates extracted from the SDF file provided by OGB")

- Note that 3D information is not provided for validation and test molecules, and test-time inference needs to be performed without explicit 3D information. Known issue: A very small number of training molecules (around 46 out of 3,378,606) have 2D graph structures that are inconsistent with the ones calculated from SMILES. These molecules often involve Si atom(s).

- Prediction task and evaluation metric: The task is graph regression: predicting the HOMO-LUMO energy gap in electronvolt (eV) given 2D molecular graphs. Mean Absolute Error (MAE) is used as evaluation metric. HOMO-LUMO gap was calculated by DFT based on the final 3D structures

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# wget http://ogb-data.stanford.edu/data/lsc/pcqm4m-v2-train.sdf.tar.gz -P /storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget # 2025-03-17 14:50:51 (14.1 MB/s) - ‘/storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget/pcqm4m-v2-train.sdf.tar.gz’ saved [1559712928/1559712928]
# md5sum /storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget/pcqm4m-v2-train.sdf.tar.gz # fd72bce606e7ddf36c2a832badeec6ab  Check against this <-
# tar -xf /storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget/pcqm4m-v2-train.sdf.tar.gz # extracts pcqm4m-v2-train.sdf
# 'fd72bce606e7ddf36c2a832badeec6ab' ==  'fd72bce606e7ddf36c2a832badeec6ab'

In [ ]:
from rdkit import Chem as rdChem
suppl = rdChem.SDMolSupplier('/storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset/from_wget/pcqm4m-v2-train.sdf')
len(suppl)

In [ ]:
from ogb.lsc import PCQM4Mv2Dataset
dataset = PCQM4Mv2Dataset(root = '/storage_common/nobilm/pretrain_paper/PCQM4Mv2Dataset', only_smiles = True)
print(len(dataset))
split_dict = dataset.get_idx_split()
len(split_dict['train']) == len(suppl)

In [ ]:
train_idxs = dataset.get_idx_split()['train']
train_dset = [dataset[i] for i in train_idxs]


In [ ]:
train_dset[2][1]


In [ ]:
i = -2
smi = train_dset[i][0]
print(smi, train_dset[i][1]) # ('CC(NCC[C@H]([C@@H]1CCC(=CC1)C)C)C', 6.811009678015001)
rdChem.MolFromSmiles(smi)

In [ ]:
smi2 = print(rdChem.MolToSmiles(suppl[i]))
suppl[i]

In [ ]:
print(smi == smi2)
from source.utils.mol_utils import visualize_3d_mols
visualize_3d_mols([suppl[i], rdChem.MolFromSmiles(smi)],width=2000)

# stuff

In [ ]:
# 500k:len(suppl) = x :100
perc = int(500000*100/len(suppl))
print(perc) # perc to be used for split
testing_data = int(perc*len(suppl)/100)
testing_data

In [ ]:
# TEST 3d
mols = []
for idx, mol in enumerate(suppl):
    print(f'{idx}-th rdkit mol obj: {mol}')
    mols.append(mol)
    if idx == 10: break

from source.utils.mol_utils import visualize_3d_mols
visualize_3d_mols(mols,width=2000, grid=(2,5))